In [18]:
!pip install spacy
!pip install nltk
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

     --------------------------------------- 12.0/12.0 MB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-py3-none-any.whl size=12019105 sha256=fd93de533815c3913b0cf873df30f6452b5bd0612abd6f49bbec9321894e66eb
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\02\87\47\4d729a97cc46afa46135595b4de32d01461f05947df39166d7
Successfully built en-core-web-sm


In [26]:
# NLTK downloads
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:
# Imports
from nltk.sentiment import SentimentIntensityAnalyzer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import en_core_web_sm

In [3]:
# Text we want to analyze
text = "Hey there! Thank you for spending some time with me! I hope we can get to know each other better!"

In [24]:
# Puncuation tokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [4]:
# VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

In [5]:
# Get the sentiment of the text
sents = sia.polarity_scores(text)

In [6]:
sents

{'neg': 0.0, 'neu': 0.636, 'pos': 0.364, 'compound': 0.8472}

In [7]:
# Summary function
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary

In [19]:
# Summarize the text
nlp = en_core_web_sm.load()
summary = summarize(text, 0.5)

In [20]:
summary

'I hope we can get to know each other better!'

In [27]:
# Remove stopwords and puncuation from the summary
filtered = [word for word in tokenizer.tokenize(summary) if word not in stopwords.words('english')]
" ".join(filtered)

'I hope get know better'